In [1]:
import numpy as np
from src.game.game_rules import move_with_chance, generate_environment, is_terminal


class Game:

    def __init__(self,
                 size: int = 5,
                 player_pos: tuple = (),
                 num_food: int = -1,
                 percent_nuke: float = 0.7,
                 intended_action_prob: float = 0.75, ):
        self.actions: tuple = ("UP", "DOWN", "LEFT", "RIGHT")
        self.environment: np.ndarray = generate_environment(size,
                                                self.actions,
                                                player_pos,
                                                num_food,
                                                percent_nuke
                                                )
        print(f"Environment:\n{self.get_styled_environment()}")
        self.size: int = size
        self.intended_action_prob: float = intended_action_prob

    def play(self):
        print(f"Initial Environment: \n {self.get_styled_environment()}\n")
        i = 0
        while (result := is_terminal(self.environment)) == "":
            i += 1
            action: str = self.get_action()
            is_intended_action  = move_with_chance(self.environment, action, self.actions, self.intended_action_prob)
            print(f"{i}.\n{self.get_styled_environment()}")
            print(("Intended Move" if is_intended_action else "Unintended Move") + "\n")
        print(f"YOU {result} after {i} moves" )

    def get_action(self) -> str:
        return input("Enter a move: ")

    def get_styled_environment(self):
        return np.array([[cell.value for cell in row] for row in self.environment])

In [2]:
game = Game(
        size=5,
        player_pos=(0, 0),
        num_food=5,
        percent_nuke= 0.7,
        intended_action_prob=1
    )

Environment Generated with: 
  - Player at position (0, 0)
  - 5 pieces of Food
  - 8 nukes

Environment:
[[' () ' 'NUKE' '----' '----' '----']
 ['----' 'FOOD' 'NUKE' '----' 'FOOD']
 ['NUKE' '----' '----' '----' 'NUKE']
 ['NUKE' 'FOOD' 'NUKE' '----' 'NUKE']
 ['FOOD' '----' 'NUKE' '----' 'FOOD']]


In [9]:
from src.game.tiles import Tiles
import torch


def get_one_hot(environment: np.ndarray) -> torch.Tensor:
    return torch.from_numpy(np.stack([
        (environment == Tiles.EMPTY),
        (
                (environment == Tiles.PLAYER) |
                (environment == Tiles.PLAYER_NUKE) |
                (environment == Tiles.PLAYER_FOOD)
        ),
        (environment == Tiles.FOOD),
        (environment == Tiles.NUKE)
    ], axis = 0)).to(torch.float32)

In [10]:
get_one_hot(game.environment)

tensor([[[0., 0., 1., 1., 1.],
         [1., 0., 0., 1., 0.],
         [0., 1., 1., 1., 0.],
         [0., 0., 0., 1., 0.],
         [0., 1., 0., 1., 0.]],

        [[1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 1.],
         [0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1.]],

        [[0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 1.],
         [1., 0., 1., 0., 1.],
         [0., 0., 1., 0., 0.]]])